In [11]:
# import necessry libraries
import sys
import os
import msgpack
import zlib
import numpy as np
import datetime
import matplotlib.pyplot as plt
import matplotlib.dates as md
from pathlib import Path

In [5]:
# import files from src
sys.path.insert(0, "..project-2/src")


example_helper = 'eh'
analysis_helper = 'ah'

In [12]:
# define the location of the input file
filename_augmento_topics = Path("..Project-2/Project_2/augmento_topics.msgpack.zlib")
filename_augmento_data = Path("../Project-2/Project_augmento_data.msgpack.zlib")
filename_bitmex_data = Path("..Project-2/Project_2/eth_data.msgpack.zlib")

In [13]:
isExist = os.path.exists(filename_augmento_data)

print(isExist)

False


In [ ]:
def load_example_data(filename_augmento_topics,
                      filename_augmento_data,
                      filename_bitmex_data,
                      datetime_start=None,
                      datetime_end=None):

	# load the topics
	with open(filename_augmento_topics, "rb") as f:
		temp = msgpack.unpackb(zlib.decompress(f.read()), encoding='utf-8')
		augmento_topics = {int(k) : v for k, v in temp.items()}
		augmento_topics_inv = {v : int(k) for k, v in temp.items()}
	
	# load the augmento data
	with open(filename_augmento_data, "rb") as f:
		temp = msgpack.unpackb(zlib.decompress(f.read()), encoding='utf-8')
		t_aug_data = np.array([el["t_epoch"] for el in temp], dtype=np.float64)
		aug_data = np.array([el["counts"] for el in temp], dtype=np.int32)
	
	# load the price data
	with open(filename_bitmex_data, "rb") as f:
		temp = msgpack.unpackb(zlib.decompress(f.read()), encoding='utf-8')
		t_price_data = np.array([el["t_epoch"] for el in temp], dtype=np.float64)
		#price_data = np.array([el["open"] for el in temp], dtype=np.float64)
		price_data = np.array([el["close"] for el in temp], dtype=np.float64)
	
	# set the start and end times if they are specified
	if datetime_start != None:
		t_start = dh.datetime_to_epoch(datetime_start)
	else:
		t_start = max(np.min(t_aug_data), np.min(t_price_data))
	
	if datetime_end != None:
		t_end = dh.datetime_to_epoch(datetime_end)
	else:
		t_end = min(np.max(t_aug_data), np.max(t_price_data))
	
	# strip the sentiments and prices outside the shared time range
	t_aug_data, aug_data = strip_data_by_time(t_aug_data, aug_data, t_start, t_end)
	t_price_data, price_data = strip_data_by_time(t_price_data, price_data, t_start, t_end)
	
	return augmento_topics, augmento_topics_inv, t_aug_data, aug_data, t_price_data, price_data

In [ ]:
# load the example data
all_data = .load_example_data(filename_augmento_topics,
                             filename_augmento_data,
                             filename_bitmex_data)
aug_topics, aug_topics_inv, t_aug_data, aug_data, t_price_data, price_data = all_data
